# A Network Intrusion Detection Method Based on Stacked Autoencoder and LSTM

In [1]:
!pip install -q sklearn
!pip install pyarrow
!pip install --upgrade pathos

     |████████████████████████████████| 17.7 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 108 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.2 MB/s  eta 0:00:01
     |████████████████████████████████| 81 kB 11.5 MB/s eta 0:00:01
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1


In [2]:
import pandas as pd
import numpy as np
import os

DIR = './Datasets/local/'

dfs = {}
names = []
for filename in os.listdir(DIR):
    dfs[filename] = pd.read_feather(DIR + filename)
    dfs[filename].columns = dfs[filename].columns.str.replace(' ', '_')
    names.append(filename)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
dataframe = dfs["cicdos2017"]
scaler = MinMaxScaler()

y = dataframe['label']
X = scaler.fit_transform(dataframe.drop(columns=['label']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [4]:
from tensorflow import expand_dims
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
    layers.Dropout(.5),
    layers.Dense(17, activation='relu'),
    layers.Dropout(.5),
    layers.Dense(7, activation='relu'),
    layers.Dropout(.5),
    layers.Reshape((-1, 7), input_shape=(7,)),
    layers.LSTM(5, return_sequences = True, activation='tanh'),
    layers.Dense(4, activation='relu'),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [17]:
model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)

In [18]:
print('\t\tgelukt!' + str(int(accuracy*10000)/100) + '%' +  ' ' + str(int(f1_score*10000)/100) + '%')

		gelukt!98.46% 94.52%


In [5]:
checkpoint_path = "./checkpoints/"
# Create a callback that saves the model's weights every epoch
cp_callbacks = [ tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', 
                    patience=20, 
                    min_delta=0.0001)
               ]
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [6]:
from matplotlib import pyplot as plt

def train_model(X_train, y_train, X_test, y_test, X_val, y_val):
    model = keras.Sequential([
        layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
        layers.Dropout(.5),
        layers.Dense(17, activation='relu'),
        layers.Dropout(.5),
        layers.Dense(7, activation='relu'),
        layers.Dropout(.5),
        layers.Reshape((-1, 7), input_shape=(7,)),
        layers.LSTM(5, return_sequences = True, activation='tanh'),
        layers.Dense(4, activation='relu'),
        layers.Dense(2, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)
    return loss, accuracy, f1_score, precision, recall

In [9]:
freq_table = {}
dss = {}
for dataset in dfs.keys():
    accuracies = {}
    results = []
    print('Start dataset:' + dataset)
    for dataset_2 in dfs.keys():
        if dataset == dataset_2:
            continue
        print('\tproberen te combineren met ' + dataset_2)
        dataframe = dfs[dataset]
        for key in results:
            dataframe.append(dfs[key])
        dataframe.append(dfs[dataset_2])
        y = dataframe['label']
        scaler = MinMaxScaler()
        df = dataframe.drop(columns=['label'])
        X = scaler.fit_transform(df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
        loss, accuracy, f1_score, precision, recall = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
        if accuracy >= 0.92:
            results.append(dataset_2)
            if dataset_2 not in freq_table:
                freq_table[dataset_2] = 0
            freq_table[dataset_2] += 1
            print('\t\tgelukt! (nr.' + str(len(results)) + ') ' + str(int(accuracy*10000)/100) + '%' +  ' ' + str(int(f1_score*10000)/100) + '%')
        else:
            print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
        accuracies[dataset_2] = tuple([dataset_2, accuracy, loss])
        
    dss[dataset] = accuracies
    if len(results) > 0:
        if dataset not in freq_table:
            freq_table[dataset] = 0
        freq_table[dataset]+=1
    break;

Start dataset:Friday-WorkingHours-Afternoon-DDoS
	proberen te combineren met Friday-WorkingHours-Afternoon-PortScan
		gelukt! (nr.1) 99.91% 99.92%
	proberen te combineren met Friday-WorkingHours-Morning-Botnet


KeyboardInterrupt: 

In [8]:
freq_table = {}
dss = {}
for dataset in dfs.keys():
    accuracies = {}
    results = []
    print('Start dataset:' + dataset)
    for dataset_2 in dfs.keys():
        if dataset == dataset_2:
            continue
        print('\tproberen te combineren met ' + dataset_2)
        dataframe = dfs[dataset]
        for key in results:
            dataframe.append(dfs[key])
        dataframe.append(dfs[dataset_2])
        y = dataframe['label']
        scaler = MinMaxScaler()
        df = dataframe.drop(columns=['label'])
        X = scaler.fit_transform(df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
        loss, accuracy, f1_score, precision, recall = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
        if accuracy >= 0.92:
            results.append(dataset_2)
            if dataset_2 not in freq_table:
                freq_table[dataset_2] = 0
            freq_table[dataset_2] += 1
            print('\t\tgelukt! (nr.' + str(len(results)) + ') ' + str(int(accuracy*10000)/100) + '%')
        else:
            print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
        accuracies[dataset_2] = tuple([dataset_2, accuracy, loss])
        
    dss[dataset] = accuracies
    if len(results) > 0:
        if dataset not in freq_table:
            freq_table[dataset] = 0
        freq_table[dataset]+=1

Start dataset:Friday-WorkingHours-Afternoon-DDoS
	proberen te combineren met Friday-WorkingHours-Afternoon-PortScan
		gelukt! (nr.1) 99.91%
	proberen te combineren met Friday-WorkingHours-Morning-Botnet
		gelukt! (nr.2) 99.93%
	proberen te combineren met Thursday-WorkingHours-Afternoon-Infiltration
		gelukt! (nr.3) 99.87%
	proberen te combineren met Thursday-WorkingHours-Morning-WebAttacks
		gelukt! (nr.4) 99.89%
	proberen te combineren met Tuesday-WorkingHours-Bruteforce
		gelukt! (nr.5) 99.85%
	proberen te combineren met Wednesday-workingHours-DoS
		gelukt! (nr.6) 98.12%
	proberen te combineren met cicdos2017
		gelukt! (nr.7) 99.49%
	proberen te combineren met Friday-02-03-2018_TrafficForML_CICFlowMeter
		gelukt! (nr.8) 99.94%
	proberen te combineren met Friday-16-02-2018_TrafficForML_CICFlowMeter
		gelukt! (nr.9) 99.92%
	proberen te combineren met 01_12_DrDoS_DNS
		gelukt! (nr.10) 99.9%
	proberen te combineren met 01_12_Syn
		gelukt! (nr.11) 99.81%
	proberen te combineren met Friday

## 2030 minuten CPU-tijd

In [28]:
acc_mergen = {}
val_loss_mergen = {}

acc_combineerbaarheid = {}
val_loss_combineerbaarheid = {}
items = 0
for dataset in dfs.keys():
    accuracy = 0
    items = 0
    val_loss = 0
    for key, item in dss[dataset].items():
        items+=1
        accuracy += item[1]
        val_loss += item[2]
        if item[0] not in acc_combineerbaarheid:
            acc_combineerbaarheid[item[0]] = 0
            val_loss_combineerbaarheid[item[0]] = 0
        acc_combineerbaarheid[item[0]] += item[1]
        val_loss_combineerbaarheid[item[0]] += item[2]
        #print("\t"+key+" :")
        #print("\t\t"+str(item))
    acc_mergen[dataset] = accuracy/items
    val_loss_mergen[dataset] = val_loss/items
beste_acc_mergen = 0
beste_val_loss_mergen = 1
beste_acc_combineerbaarheid = 0
beste_val_loss_combineerbaarheid = 1

beste_acc_combineerbaarheid_dataset = ""
beste_val_loss_combineerbaarheid_dataset = ""
beste_acc_mergen_dataset = ""
beste_val_loss_mergen_dataset = ""

gem_acc_mergen = 0
gem_val_loss_mergen = 0
gem_acc_combineerbaarheid = 0
gem_val_loss_combineerbaarheid = 0

for dataset in dfs.keys():
    acc_combineerbaarheid[dataset] = acc_combineerbaarheid[dataset]/items
    val_loss_combineerbaarheid[dataset] = val_loss_combineerbaarheid[dataset]/items
    gem_acc_combineerbaarheid += acc_combineerbaarheid[dataset]
    gem_val_loss_combineerbaarheid += val_loss_combineerbaarheid[dataset]
    gem_acc_mergen += acc_mergen[dataset]
    gem_val_loss_mergen += val_loss_mergen[dataset]
    if acc_combineerbaarheid[dataset] > beste_acc_combineerbaarheid:
        beste_acc_combineerbaarheid_dataset = dataset
        beste_acc_combineerbaarheid = acc_combineerbaarheid[dataset]
        
    if val_loss_combineerbaarheid[dataset] < beste_val_loss_combineerbaarheid:
        beste_val_loss_combineerbaarheid_dataset = dataset
        beste_val_loss_combineerbaarheid = val_loss_combineerbaarheid[dataset]
        
    if acc_mergen[dataset] > beste_acc_mergen:
        beste_acc_mergen_dataset = dataset
        beste_acc_mergen = acc_mergen[dataset]
        
    if val_loss_mergen[dataset] < beste_val_loss_mergen:
        beste_val_loss_mergen_dataset = dataset
        beste_val_loss_mergen = val_loss_mergen[dataset]
    #print(dataset)
    #print("\taccuracy-combineerbaarheid:")
    #print("\t\t" + str(acc_combineerbaarheid[dataset]))
    #print("\tval_loss-combineerbaarheid:")
    #print("\t\t" + str(val_loss_combineerbaarheid[dataset]))
    #print("\taccuracy-mergen:")
    #print("\t\t" + str(acc_mergen[dataset]))
    #print("\tval_loss-mergen:")
    #print("\t\t" + str(val_loss_mergen[dataset]))
gem_acc_combineerbaarheid /= items
gem_val_loss_combineerbaarheid /= items
gem_acc_mergen /= items
gem_val_loss_mergen /= items
print("dataset met beste accuracy-combineerbaarheid: " + beste_acc_combineerbaarheid_dataset + " " + str(int(beste_acc_combineerbaarheid*10000)/100))
print("dataset met beste val_loss-combineerbaarheid: " + beste_val_loss_combineerbaarheid_dataset + " " + str(int(beste_val_loss_combineerbaarheid*10000)/100))
print("dataset met beste accuracy-mergen: " + beste_acc_mergen_dataset + " " + str(int(beste_acc_mergen*10000)/100))
print("dataset met beste val_loss-mergen: " + beste_val_loss_mergen_dataset + " " + str(int(beste_val_loss_mergen*10000)/100))
print("gemiddelde accuracy mergen: " + str(int(gem_acc_mergen*10000)/100))
print("gemiddelde val_loss mergen: " + str(int(gem_val_loss_mergen*10000)/100))
print("gemiddelde accuracy combineerbaarheid: " + str(int(gem_acc_combineerbaarheid*10000)/100))
print("gemiddelde val_loss combineerbaarheid: " + str(int(gem_val_loss_combineerbaarheid*10000)/100))

dataset met beste accuracy-combineerbaarheid: Wednesday-28-02-2018_TrafficForML_CICFlowMeter 99.08
dataset met beste val_loss-combineerbaarheid: 03_11_UDPLag 10.37
dataset met beste accuracy-mergen: Wednesday-21-02-2018_TrafficForML_CICFlowMeter 99.99
dataset met beste val_loss-mergen: Thursday-WorkingHours-Afternoon-Infiltration 0.13
gemiddelde accuracy mergen: 99.62
gemiddelde val_loss mergen: 14.51
gemiddelde accuracy combineerbaarheid: 99.62
gemiddelde val_loss combineerbaarheid: 14.51


In [12]:
results = []
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
print('Start dataset:' + dataset)
dataframe = dfs[dataset]
trainingdataset, testingdataset = train_test_split(dfs["Wednesday-28-02-2018_TrafficForML_CICFlowMeter"], test_size=0.9)
for key in dfs.keys(): 
    if key == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        dataframe.append(trainingdataset)
    else:
        if dataset != dataset_2:
            dataframe.append(dfs[key])
y = dataframe['label']
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
model = keras.Sequential([
    layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
    layers.Dropout(.5),
    layers.Dense(17, activation='relu'),
    layers.Dropout(.5),
    layers.Dense(7, activation='relu'),
    layers.Dropout(.5),
    layers.Reshape((-1, 7), input_shape=(7,)),
    layers.LSTM(5, return_sequences = True, activation='tanh'),
    layers.Dense(4, activation='relu'),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)
if accuracy >= 0.92:
    if dataset_2 != "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        results.append(dataset_2)
    print('\t\tgelukt! (nr.' + str(f1_score) + ') ' + str(int(accuracy*10000)/100) + '%')
else:
    print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
dataframe = testingdataset
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)

Start dataset:Wednesday-21-02-2018_TrafficForML_CICFlowMeter
		gelukt! (nr.1.0) 100.0%


In [ ]:
results = []
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
print('Start dataset:' + dataset)
for dataset_2 in dfs.keys():
    if dataset == dataset_2:
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs[dataset]
    for key in results:
        dataframe.append(dfs[key])
    if dataset_2 == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        trainingdataset, testingdataset = train_test_split(dfs[dataset_2], test_size=0.9)
        dataframe.append(trainingdataset)
    else:
        dataframe.append(dfs[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    model = keras.Sequential([
        layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
        layers.Dropout(.5),
        layers.Dense(17, activation='relu'),
        layers.Dropout(.5),
        layers.Dense(7, activation='relu'),
        layers.Dropout(.5),
        layers.Reshape((-1, 7), input_shape=(7,)),
        layers.LSTM(5, return_sequences = True, activation='tanh'),
        layers.Dense(4, activation='relu'),
        layers.Dense(2, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)
    if accuracy >= 0.92:
        if dataset_2 != "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
            results.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(f1_score) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
dataframe = testingdataset
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)

In [14]:
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([546212])
y_round = list(y_round)
print(len(y_round))

546212


In [15]:
print(len(y_true))

546212


In [16]:
column_names = list(dfs[filename].columns)
confusion = tf.math.confusion_matrix(labels=y_true, predictions=y_round)
print(confusion)

tf.Tensor(
[[477120   7576]
 [ 60244   1272]], shape=(2, 2), dtype=int32)


In [17]:
from sklearn.metrics import accuracy_score
print("Accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

Accuracy van model met een ongekende dataset:
87.58%


In [18]:
from sklearn.metrics import balanced_accuracy_score
print("Gebalanceerde accuracy van model met een ongekende dataset:")
print(str(int(balanced_accuracy_score(y_true, y_round)*10000)/100) + "%")

Gebalanceerde accuracy van model met een ongekende dataset:
50.25%


In [20]:
from sklearn.metrics import f1_score
print("Gebalanceerde accuracy van model met een ongekende dataset:")
print(str(int(f1_score(y_true, y_round, average='weighted')*10000)/100) + "%")

Gebalanceerde accuracy van model met een ongekende dataset:
83.25%


In [98]:
dataframe = dfs['Wednesday-21-02-2018_TrafficForML_CICFlowMeter']

In [90]:
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([1048575])
y_round = list(y_round)

In [91]:
print("Accuracy van model met een gekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

Accuracy van model met een gekende dataset:
99.99%


In [92]:
print("Gebalanceerde accuracy van model met een gekende dataset:")
print(str(int(balanced_accuracy_score(y_true, y_round)*10000)/100) + "%")

Gebalanceerde accuracy van model met een gekende dataset:
99.99%


In [114]:
from sklearn.utils import resample
df_minority = dataframe[dataframe['label']==1]
df_majority = dataframe[dataframe['label']==0]
samples = len(df_majority)
if len(df_minority) < len(df_majority):
    samples = len(df_minority)
maj_class = resample(df_majority,replace=True,n_samples=samples,random_state=42) 
min_class = resample(df_minority,replace=True,n_samples=samples,random_state=42)
dataframe = pd.concat([maj_class,min_class])
print(dataframe['label'].value_counts())

1    360833
0    360833
Name: label, dtype: int64


In [95]:
min_class = resample(df_minority,replace=True,n_samples=1324,random_state=123) 
maj_class = resample(df_majority2,replace=True,n_samples=1324,random_state=123) 

label        
0      0         0
       1         0
       2         0
       3         0
       4         0
                ..
1      360828    1
       360829    1
       360830    1
       360831    1
       360832    1
Name: label, Length: 721666, dtype: uint8


In [115]:
from sklearn.utils import resample
dfs = {}
names = []
for filename in os.listdir(DIR):
    dfs[filename] = pd.read_feather(DIR + filename)
    dfs[filename].columns = dfs[filename].columns.str.replace(' ', '_')
    df_minority = dfs[filename][dfs[filename]['label'] == 1]
    df_majority = dfs[filename][dfs[filename]['label'] == 0]
    samples = len(df_majority)
    if len(df_minority) < len(df_majority):
        samples = len(df_minority)
    maj_class = resample(df_majority,replace=True,n_samples=samples,random_state=42) 
    min_class = resample(df_minority,replace=True,n_samples=samples,random_state=42)
    dfs[filename] = pd.concat([maj_class,min_class])
    names.append(filename)

In [117]:
results = []
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
print('Start dataset:' + dataset)
for dataset_2 in dfs.keys():
    if dataset == dataset_2 or dataset_2 == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs[dataset]
    for key in results:
        dataframe.append(dfs[key])
    dataframe.append(dfs[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    model = keras.Sequential([
        layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
        layers.Dropout(.5),
        layers.Dense(17, activation='relu'),
        layers.Dropout(.5),
        layers.Dense(7, activation='relu'),
        layers.Dropout(.5),
        layers.Reshape((-1, 7), input_shape=(7,)),
        layers.LSTM(5, return_sequences = True, activation='tanh'),
        layers.Dense(4, activation='relu'),
        layers.Dense(2, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
    loss, accuracy = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)
    if accuracy >= 0.92:
        results.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(len(results)) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
dataframe = dfs['Wednesday-28-02-2018_TrafficForML_CICFlowMeter']
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)

Start dataset:Wednesday-21-02-2018_TrafficForML_CICFlowMeter
	proberen te combineren met Friday-WorkingHours-Afternoon-DDoS
		gelukt! (nr.1) 100.0%
	proberen te combineren met Friday-WorkingHours-Afternoon-PortScan
		gelukt! (nr.2) 100.0%
	proberen te combineren met Friday-WorkingHours-Morning-Botnet
		gelukt! (nr.3) 100.0%
	proberen te combineren met Thursday-WorkingHours-Afternoon-Infiltration
		gelukt! (nr.4) 99.99%
	proberen te combineren met Thursday-WorkingHours-Morning-WebAttacks
		gelukt! (nr.5) 100.0%
	proberen te combineren met Tuesday-WorkingHours-Bruteforce
		gelukt! (nr.6) 100.0%
	proberen te combineren met Wednesday-workingHours-DoS
		gelukt! (nr.7) 99.99%
	proberen te combineren met cicdos2017
		gelukt! (nr.8) 100.0%
	proberen te combineren met Friday-02-03-2018_TrafficForML_CICFlowMeter
		gelukt! (nr.9) 100.0%
	proberen te combineren met Friday-16-02-2018_TrafficForML_CICFlowMeter
		gelukt! (nr.10) 99.99%
	proberen te combineren met 01_12_DrDoS_DNS
		gelukt! (nr.11) 99.

In [119]:
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([136472])
y_round = list(y_round)

In [120]:
from sklearn.metrics import accuracy_score
print("Accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

Accuracy van model met een ongekende dataset:
50.22%


In [121]:
print("Gebalanceerde accuracy van model met een gekende dataset:")
print(str(int(balanced_accuracy_score(y_true, y_round)*10000)/100) + "%")

Gebalanceerde accuracy van model met een gekende dataset:
50.22%


## Per aanvalsklasse

In [199]:
import pandas as pd
import numpy as np

PREPATH = "/project/L/datasets/clean-collection"
df2017 = [
    "/cicids2017-clean/Friday-WorkingHours-Afternoon-DDoS.pcap_ISCX.csv",
    "/cicids2017-clean/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "/cicids2017-clean/Friday-WorkingHours-Morning-Botnet.pcap_ISCX.csv",
    "/cicids2017-clean/Thursday-WorkingHours-Afternoon-Infiltration.pcap_ISCX.csv",
    "/cicids2017-clean/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "/cicids2017-clean/Tuesday-WorkingHours-Bruteforce.pcap_ISCX.csv",
    "/cicids2017-clean/Wednesday-workingHours-DoS.pcap_ISCX.csv"
]
df2017_2 = [
    "/cicdos2017-clean/cicdos2017.csv"
]
df2018 = [
    "/cicids2018-clean/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Tuesday-20-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv"
]

df2019 = [
    "/cicddos2019-clean/01_12_DrDoS_DNS.csv",
    "/cicddos2019-clean/01_12_DrDoS_LDAP.csv",
    "/cicddos2019-clean/01_12_DrDoS_MSSQL.csv",
    "/cicddos2019-clean/01_12_DrDoS_NetBIOS.csv",
    "/cicddos2019-clean/01_12_DrDoS_NTP.csv",
    "/cicddos2019-clean/01_12_DrDoS_SNMP.csv",
    "/cicddos2019-clean/01_12_DrDoS_SSDP.csv",
    "/cicddos2019-clean/01_12_DrDoS_UDP.csv",
    "/cicddos2019-clean/01_12_Syn.csv",
    "/cicddos2019-clean/01_12_TFTP.csv",
    "/cicddos2019-clean/01_12_UDPLag.csv",
    "/cicddos2019-clean/03_11_LDAP.csv",
    "/cicddos2019-clean/03_11_MSSQL.csv",
    "/cicddos2019-clean/03_11_NetBIOS.csv",
    "/cicddos2019-clean/03_11_Portmap.csv",
    "/cicddos2019-clean/03_11_Syn.csv",
    "/cicddos2019-clean/03_11_UDP.csv",
    "/cicddos2019-clean/03_11_UDPLag.csv"
]
dfs_names = [df2017, df2017_2, df2018, df2019]

In [207]:
dfs_raw = {}
for dataset in dfs_names:
    for path in dataset:
        filename = path.split('/')[2].split('.')[0]
        dfs_raw[filename] = pd.read_csv(f'{PREPATH}{path}', low_memory=False, header=0, names= ["destination port", "protocol", "timestamp", "flow duration", "total fwd packets", "total backward packets", "total length of fwd packets", "total length of bwd packets", "fwd packet length max", "fwd packet length min", "fwd packet length mean", "fwd packet length std", "bwd packet length max", "bwd packet length min", "bwd packet length mean", "bwd packet length std", "flow bytes/s", "flow packets/s", "flow iat mean", "flow iat std", "flow iat max", "flow iat min", "fwd iat total", "fwd iat mean", "fwd iat std", "fwd iat max", "fwd iat min", "bwd iat total", "bwd iat mean", "bwd iat std", "bwd iat max", "bwd iat min", "fwd psh flags", "bwd psh flags", "fwd urg flags", "bwd urg flags", "fwd header length", "bwd header length", "fwd packets/s", "bwd packets/s", "min packet length", "max packet length", "packet length mean", "packet length std", "packet length variance", "fin flag count", "syn flag count", "rst flag count", "psh flag count", "ack flag count", "urg flag count", "cwe flag count", "ece flag count", "down/up ratio", "average packet size", "avg fwd segment size", "avg bwd segment size", "fwd avg bytes/bulk", "fwd avg packets/bulk", "fwd avg bulk rate", "bwd avg bytes/bulk", "bwd avg packets/bulk", "bwd avg bulk rate", "subflow fwd packets", "subflow fwd bytes", "subflow bwd packets", "subflow bwd bytes", "init_win_bytes_forward", "init_win_bytes_backward", "act_data_pkt_fwd", "min_seg_size_forward", "active mean", "active std", "active max", "active min", "idle mean", "idle std", "idle max", "idle min", "label"])
        dfs_raw[filename].columns = dfs_raw[filename].columns.str.replace(' ', '_')

In [208]:
import re

regex_patDOS = re.compile(r'.*dos.*', flags=re.IGNORECASE)
regex_patBruteForce = re.compile(r'.*brute.*', flags=re.IGNORECASE)
for i, path in dfs_raw.items():
    dfs_raw[i] = dfs_raw[i].replace(['slowloris','hulk','Bot','slowbody2','slowread','goldeneye','slowheaders','rudy','TFTP','Syn','UDP-lag','Portmap','UDP','UDPLag'], "DoS")
    dfs_raw[i] = dfs_raw[i].replace(regex_patBruteForce, "BruteForce")
    dfs_raw[i] = dfs_raw[i].replace(regex_patDOS, "DoS")
    dfs_raw[i] = dfs_raw[i].replace(['SQL Injection','NetBIOS','LDAP','MSSQL','Heartbleed','FTP-Patator','SSH-Patator'], "BruteForce")
    dfs_raw[i] = dfs_raw[i].replace(['.*XSS.*','.*Sql.*'], "DoS",regex=True)
    dfs_raw[i] = dfs_raw[i].replace(['BENIGN','Benign'], "normal")
    dfs_raw[i] = dfs_raw[i][dfs_raw[i].label != 'Infiltration']
    dfs_raw[i] = dfs_raw[i][dfs_raw[i].label != 'Infilteration']

In [209]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object and props[col].dtype != 'datetime64[ns]':  # Exclude strings
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)
            else:
                props[col] = props[col].astype(np.float32)
    mem_usg = props.memory_usage().sum() / 1024**2 
    return props, NAlist

In [210]:
for i, path in dfs_raw.items():
    dfs_raw[i] = dfs_raw[i].drop(["destination_port", "protocol", "timestamp"], axis=1)
    dfs_raw[i] = dfs_raw[i].replace(["Infinity", "Inf"], np.nan)
    float_cols = list(dfs_raw[i].select_dtypes(include='float64').columns)
    dfs_raw[i][float_cols] = dfs_raw[i][float_cols].clip(lower=np.finfo(np.float32).min, upper=np.finfo(np.float32).max)
    dfs_raw[i][float_cols] = dfs_raw[i][float_cols].astype(dtype='float32')
    dfs_raw[i], NAlist = reduce_mem_usage(dfs_raw[i])

In [211]:
from sklearn import preprocessing
attack_types = []
for i, path in dfs_raw.items():
    attack_types = list(dfs_raw[i]["label"].unique()) + attack_types
u, indices = np.unique(attack_types, return_inverse=True)

In [212]:
print(u)

['BruteForce' 'DoS' 'PortScan' 'normal']


In [213]:
label_encoder = preprocessing.LabelEncoder()
label_encoder = label_encoder.fit(u)

for i, path in dfs_raw.items():
    dfs_raw[i]['label'] = label_encoder.transform(dfs_raw[i]['label'])
    print(i)

Friday-WorkingHours-Afternoon-DDoS
Friday-WorkingHours-Afternoon-PortScan
Friday-WorkingHours-Morning-Botnet
Thursday-WorkingHours-Afternoon-Infiltration
Thursday-WorkingHours-Morning-WebAttacks
Tuesday-WorkingHours-Bruteforce
Wednesday-workingHours-DoS
cicdos2017
Friday-02-03-2018_TrafficForML_CICFlowMeter
Friday-16-02-2018_TrafficForML_CICFlowMeter
Friday-23-02-2018_TrafficForML_CICFlowMeter
Thursday-01-03-2018_TrafficForML_CICFlowMeter
Thursday-15-02-2018_TrafficForML_CICFlowMeter
Thursday-22-02-2018_TrafficForML_CICFlowMeter
Tuesday-20-02-2018_TrafficForML_CICFlowMeter
Wednesday-14-02-2018_TrafficForML_CICFlowMeter
Wednesday-21-02-2018_TrafficForML_CICFlowMeter
Wednesday-28-02-2018_TrafficForML_CICFlowMeter
01_12_DrDoS_DNS
01_12_DrDoS_LDAP
01_12_DrDoS_MSSQL
01_12_DrDoS_NetBIOS
01_12_DrDoS_NTP
01_12_DrDoS_SNMP
01_12_DrDoS_SSDP
01_12_DrDoS_UDP
01_12_Syn
01_12_TFTP
01_12_UDPLag
03_11_LDAP
03_11_MSSQL
03_11_NetBIOS
03_11_Portmap
03_11_Syn
03_11_UDP
03_11_UDPLag


In [12]:
def train_model(X_train, y_train, X_test, y_test, X_val, y_val):
    model = keras.Sequential([
        layers.Dense(27, input_dim=X_train.shape[1], activation='relu'),
        layers.Dropout(.5),
        layers.Dense(17, activation='relu'),
        layers.Dropout(.5),
        layers.Dense(7, activation='relu'),
        layers.Dropout(.5),
        layers.Reshape((-1, 7), input_shape=(7,)),
        layers.LSTM(5, return_sequences = True, activation='tanh'),
        layers.Dense(4, activation='relu'),
        layers.Dense(2, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200, batch_size=2048, callbacks = cp_callbacks, verbose = 0)
    loss, accuracy = model.evaluate(X_test, y_test, batch_size = 2048, verbose = 0)
    return loss, accuracy

In [215]:
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
accuracies_cat = {}
results_cat = []
print('Start dataset:' + dataset)
for dataset_2 in dfs_raw.keys():
    if dataset == dataset_2  or dataset_2 == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs_raw[dataset]
    for key in results_cat:
        dataframe.append(dfs_raw[key])
    dataframe.append(dfs_raw[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    loss, accuracy = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
    if accuracy >= 0.92:
        results.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(len(results)) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
    accuracies_cat[dataset_2] = tuple([dataset_2, accuracy, loss])
dataframe = dfs_raw['Wednesday-28-02-2018_TrafficForML_CICFlowMeter']
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)

Start dataset:Wednesday-21-02-2018_TrafficForML_CICFlowMeter
	proberen te combineren met Friday-WorkingHours-Afternoon-DDoS
		 niet gelukt, accuracy was 65.54%
	proberen te combineren met Friday-WorkingHours-Afternoon-PortScan
		 niet gelukt, accuracy was 65.71%
	proberen te combineren met Friday-WorkingHours-Morning-Botnet
		 niet gelukt, accuracy was 0.13%
	proberen te combineren met Thursday-WorkingHours-Afternoon-Infiltration
		 niet gelukt, accuracy was 65.61%
	proberen te combineren met Thursday-WorkingHours-Morning-WebAttacks
		 niet gelukt, accuracy was 65.5%
	proberen te combineren met Tuesday-WorkingHours-Bruteforce
		 niet gelukt, accuracy was 0.15%
	proberen te combineren met Wednesday-workingHours-DoS
		 niet gelukt, accuracy was 65.72%
	proberen te combineren met cicdos2017
		 niet gelukt, accuracy was 0.0%
	proberen te combineren met Friday-02-03-2018_TrafficForML_CICFlowMeter
		 niet gelukt, accuracy was 0.0%
	proberen te combineren met Friday-16-02-2018_TrafficForML_CI

## Accuracies

In [ ]:
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([1152382])
y_round = list(y_round)
print(len(y_round))

In [20]:
y_true = dataframe['label']
print(len(y_true))

1375484


In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

In [219]:
print("Gebalanceerde accuracy van model met een gekende dataset:")
print(str(int(balanced_accuracy_score(y_true, y_round)*10000)/100) + "%")

Gebalanceerde accuracy van model met een gekende dataset:
0.0%


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Dit is natuurlijk zeer slecht, misschien is de aanvalsklasse verkeerd gemapped uit de oorspronkelijke aanvalsklasse?

Een andere, iets minder ambitieuse, strategie in de volgende sectie.

# één aanvalsklasse beschouwen

##### voor elke klasse testen
#### vereiste: minstens 50 samples van een aanvalsklasse (na balancering) of de dataset kan niet gebruikt worden

In [19]:
import pandas as pd
import numpy as np

PREPATH = "/project/L/datasets/ids-clean-data-collection"
df2017 = [
    "/cicids2017-clean/Friday-WorkingHours-Afternoon-DDoS.pcap_ISCX.csv",
    "/cicids2017-clean/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "/cicids2017-clean/Friday-WorkingHours-Morning-Botnet.pcap_ISCX.csv",
    "/cicids2017-clean/Thursday-WorkingHours-Afternoon-Infiltration.pcap_ISCX.csv",
    "/cicids2017-clean/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "/cicids2017-clean/Tuesday-WorkingHours-Bruteforce.pcap_ISCX.csv",
    "/cicids2017-clean/Wednesday-workingHours-DoS.pcap_ISCX.csv"
]
df2017_2 = [
    "/cicdos2017-clean/cicdos2017.csv"
]
df2018 = [
    "/cicids2018-clean/Botnet-Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/DoS2-Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Web2-Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Infil2-Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/DoS1-Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Web1-Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Bruteforce-Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/DDoS2-Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    "/cicids2018-clean/Infil1-Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv"
]

df2019 = [
    "/cicddos2019-clean/01_12_DrDoS_DNS.csv",
    "/cicddos2019-clean/01_12_DrDoS_LDAP.csv",
    "/cicddos2019-clean/01_12_DrDoS_MSSQL.csv",
    "/cicddos2019-clean/01_12_DrDoS_NetBIOS.csv",
    "/cicddos2019-clean/01_12_DrDoS_NTP.csv",
    "/cicddos2019-clean/01_12_DrDoS_SNMP.csv",
    "/cicddos2019-clean/01_12_DrDoS_SSDP.csv",
    "/cicddos2019-clean/01_12_DrDoS_UDP.csv",
    "/cicddos2019-clean/01_12_Syn.csv",
    "/cicddos2019-clean/01_12_TFTP.csv",
    "/cicddos2019-clean/01_12_UDPLag.csv",
    "/cicddos2019-clean/03_11_LDAP.csv",
    "/cicddos2019-clean/03_11_MSSQL.csv",
    "/cicddos2019-clean/03_11_NetBIOS.csv",
    "/cicddos2019-clean/03_11_Portmap.csv",
    "/cicddos2019-clean/03_11_Syn.csv",
    "/cicddos2019-clean/03_11_UDP.csv",
    "/cicddos2019-clean/03_11_UDPLag.csv"
]
dfs_names = [df2017, df2017_2, df2018, df2019]

In [21]:
dfs_raw = {}
for dataset in dfs_names:
    for path in dataset:
        filename = path.split('/')[2].split('.')[0]
        dfs_raw[filename] = pd.read_csv(f'{PREPATH}{path}', low_memory=False, header=0, names= ["destination port", "protocol", "timestamp", "flow duration", "total fwd packets", "total backward packets", "total length of fwd packets", "total length of bwd packets", "fwd packet length max", "fwd packet length min", "fwd packet length mean", "fwd packet length std", "bwd packet length max", "bwd packet length min", "bwd packet length mean", "bwd packet length std", "flow bytes/s", "flow packets/s", "flow iat mean", "flow iat std", "flow iat max", "flow iat min", "fwd iat total", "fwd iat mean", "fwd iat std", "fwd iat max", "fwd iat min", "bwd iat total", "bwd iat mean", "bwd iat std", "bwd iat max", "bwd iat min", "fwd psh flags", "bwd psh flags", "fwd urg flags", "bwd urg flags", "fwd header length", "bwd header length", "fwd packets/s", "bwd packets/s", "min packet length", "max packet length", "packet length mean", "packet length std", "packet length variance", "fin flag count", "syn flag count", "rst flag count", "psh flag count", "ack flag count", "urg flag count", "cwe flag count", "ece flag count", "down/up ratio", "average packet size", "avg fwd segment size", "avg bwd segment size", "fwd avg bytes/bulk", "fwd avg packets/bulk", "fwd avg bulk rate", "bwd avg bytes/bulk", "bwd avg packets/bulk", "bwd avg bulk rate", "subflow fwd packets", "subflow fwd bytes", "subflow bwd packets", "subflow bwd bytes", "init_win_bytes_forward", "init_win_bytes_backward", "act_data_pkt_fwd", "min_seg_size_forward", "active mean", "active std", "active max", "active min", "idle mean", "idle std", "idle max", "idle min", "label"])
        dfs_raw[filename].columns = dfs_raw[filename].columns.str.replace(' ', '_')

In [22]:
import re

regex_patDOS = re.compile(r'.*dos.*', flags=re.IGNORECASE)
regex_patBruteForce = re.compile(r'.*brute.*', flags=re.IGNORECASE)
for i, path in dfs_raw.items():
    dfs_raw[i] = dfs_raw[i].replace(['slowloris','hulk','Bot','slowbody2','slowread','goldeneye','slowheaders','rudy','TFTP','Syn','UDP-lag','Portmap','UDP'], "DDoS")
    dfs_raw[i] = dfs_raw[i].replace(regex_patDOS, "DDoS")
    dfs_raw[i] = dfs_raw[i].replace(['UDPLag'], "DoS")
    dfs_raw[i] = dfs_raw[i].replace(regex_patBruteForce, "BruteForce")
    dfs_raw[i] = dfs_raw[i].replace(['SQL Injection','NetBIOS','LDAP','MSSQL','Heartbleed','FTP-Patator','SSH-Patator'], "Code_Injection")
    dfs_raw[i] = dfs_raw[i].replace(['.*XSS.*','.*Sql.*'], "Code_Injection",regex=True)
    dfs_raw[i] = dfs_raw[i].replace(['BENIGN','Benign'], "normal")
    dfs_raw[i] = dfs_raw[i][dfs_raw[i].label != 'Infiltration']
    dfs_raw[i] = dfs_raw[i][dfs_raw[i].label != 'Infilteration']

In [23]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object and props[col].dtype != 'datetime64[ns]':  # Exclude strings
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)
            else:
                props[col] = props[col].astype(np.float32)
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [24]:
for i, path in dfs_raw.items():
    dfs_raw[i] = dfs_raw[i].drop(["destination_port", "protocol", "timestamp"], axis=1)
    dfs_raw[i] = dfs_raw[i].replace(["Infinity", "Inf"], np.nan)
    float_cols = list(dfs_raw[i].select_dtypes(include='float64').columns)
    dfs_raw[i][float_cols] = dfs_raw[i][float_cols].clip(lower=np.finfo(np.float32).min, upper=np.finfo(np.float32).max)
    dfs_raw[i][float_cols] = dfs_raw[i][float_cols].astype(dtype='float32')
    dfs_raw[i], NAlist = reduce_mem_usage(dfs_raw[i])

Memory usage of properties dataframe is : 113.65452575683594  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  50.369619369506836  MB
This is  44.31818181818182 % of the initial size
Memory usage of properties dataframe is : 144.060791015625  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  64.93649291992188  MB
This is  45.07575757575758 % of the initial size
Memory usage of properties dataframe is : 96.13133239746094  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  45.15259552001953  MB
This is  46.96969696969697 % of the initial size
Memory usage of properties dataframe is : 145.20030212402344  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  65.45013618469238  MB
This is  45.07575757575758 % of the initial size
Memory usage of properties dataframe is : 85.71812438964844  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  40.26154327392578  MB
This is  46.96969696969697 % of the initial size
Memory usage of properties dataframe is : 224.40

In [33]:
from sklearn.utils import resample

dfs_dos = {}
dfs_ddos = {}
dfs_brute = {}
dfs_port = {}
dfs_normal = {}
normal_class_port = {}
normal_class_brute = {}
normal_class_dos = {}
normal_class_ddos = {}
dataset_counter_dos = 0
dataset_counter_ddos = 0
dataset_counter_brute = 0
dataset_counter_port = 0

datasets_dos = []
datasets_ddos = []
datasets_brute = []
datasets_port = []

for filename, path in dfs_raw.items():
    dfs_normal[filename] = dfs_raw[filename][dfs_raw[filename]['label'] == "normal"]
    dfs_ddos[filename] = dfs_raw[filename][dfs_raw[filename]['label'] == "DDoS"]
    dfs_dos[filename] = dfs_raw[filename][dfs_raw[filename]['label'] == "DoS"]
    dfs_brute[filename] = dfs_raw[filename][dfs_raw[filename]['label'] == "BruteForce"]
    dfs_port[filename] = dfs_raw[filename][dfs_raw[filename]['label'] == "PortScan"]
    
    # balanceren
    samples_dos = len(dfs_dos[filename])
    samples_ddos = len(dfs_ddos[filename])
    samples_brute = len(dfs_brute[filename])
    samples_port = len(dfs_port[filename])
    normal_class_dos[filename] = resample(dfs_normal[filename],replace=True,n_samples=samples_dos,random_state=42) 
    normal_class_brute[filename] = resample(dfs_normal[filename],replace=True,n_samples=samples_brute,random_state=42) 
    normal_class_port[filename] = resample(dfs_normal[filename],replace=True,n_samples=samples_port,random_state=42)
    normal_class_ddos[filename] = resample(dfs_normal[filename],replace=True,n_samples=samples_ddos,random_state=42) 

    dfs_dos[filename] = pd.concat([normal_class_dos[filename], dfs_dos[filename]])
    dfs_ddos[filename] = pd.concat([normal_class_ddos[filename], dfs_ddos[filename]])
    dfs_brute[filename] = pd.concat([normal_class_brute[filename], dfs_brute[filename]])
    dfs_port[filename] = pd.concat([normal_class_port[filename], dfs_port[filename]])
    
    dfs_ddos[filename] = dfs_ddos[filename].replace('normal', 0)
    dfs_ddos[filename] = dfs_ddos[filename].replace('DDoS', 1)
    dfs_dos[filename] = dfs_dos[filename].replace('normal', 0)
    dfs_dos[filename] = dfs_dos[filename].replace('DoS', 1)
    dfs_brute[filename] = dfs_brute[filename].replace('normal', 0)
    dfs_brute[filename] = dfs_brute[filename].replace('BruteForce', 1)
    dfs_port[filename] = dfs_port[filename].replace('normal', 0)
    dfs_port[filename] = dfs_port[filename].replace('PortScan', 1)
    print(filename+":")
    print("DDOS: ")
    if len(dfs_ddos[filename])/2 < 50:
        print("Onvoldoende samples.")
    else:
        datasets_ddos.append(filename)
        print(dfs_ddos[filename]['label'].value_counts())
        
    print("DOS: ")
    if len(dfs_dos[filename])/2 < 50:
        print("Onvoldoende samples.")
    else:
        datasets_dos.append(filename)
        print(dfs_dos[filename]['label'].value_counts())
        
    print("BRUTE: ")
    if len(dfs_brute[filename])/2 < 50:
        print("Onvoldoende samples.")
    else:
        datasets_brute.append(filename)
        print(dfs_brute[filename]['label'].value_counts())
        
    print("PORT: ")
    if len(dfs_port[filename])/2 < 50:
        print("Onvoldoende samples.")
    else:
        datasets_port.append(filename)
        print(dfs_port[filename]['label'].value_counts())
    print("-------------------------")
    
print("-----samengevat-----")

print("DDOS: ")
if len(datasets_ddos)>1:
    print("Op deze aanvalsklasse kan getrained worden")
    print(datasets_ddos)
else:
    print("Op deze aanvalsklasse zal niet getrained kunnen worden, onvoldoende datasets")
    
print("DOS: ")
if len(datasets_dos)>1:
    print("Op deze aanvalsklasse kan getrained worden")
    print(datasets_dos)
else:
    print("Op deze aanvalsklasse zal niet getrained kunnen worden, onvoldoende datasets")
    
print("BRUTE: ")
if len(datasets_brute)>1:
    print("Op deze aanvalsklasse kan getrained worden")
    print(datasets_brute)
else:
    print("Op deze aanvalsklasse zal niet getrained kunnen worden, onvoldoende datasets")
    
print("PORT: ")
if len(datasets_port)>1:
    print("Op deze aanvalsklasse kan getrained worden")
    print(datasets_port)
else:
    print("Op deze aanvalsklasse zal niet getrained kunnen worden, onvoldoende datasets")

Friday-WorkingHours-Afternoon-DDoS:
DDOS: 
1    128025
0    128025
Name: label, dtype: int64
DOS: 
Onvoldoende samples.
BRUTE: 
Onvoldoende samples.
PORT: 
Onvoldoende samples.
-------------------------
Friday-WorkingHours-Afternoon-PortScan:
DDOS: 
Onvoldoende samples.
DOS: 
Onvoldoende samples.
BRUTE: 
Onvoldoende samples.
PORT: 
1    158804
0    158804
Name: label, dtype: int64
-------------------------
Friday-WorkingHours-Morning-Botnet:
DDOS: 
1    1956
0    1956
Name: label, dtype: int64
DOS: 
Onvoldoende samples.
BRUTE: 
Onvoldoende samples.
PORT: 
Onvoldoende samples.
-------------------------
Thursday-WorkingHours-Afternoon-Infiltration:
DDOS: 
Onvoldoende samples.
DOS: 
Onvoldoende samples.
BRUTE: 
Onvoldoende samples.
PORT: 
Onvoldoende samples.
-------------------------
Thursday-WorkingHours-Morning-WebAttacks:
DDOS: 
Onvoldoende samples.
DOS: 
Onvoldoende samples.
BRUTE: 
1    1507
0    1507
Name: label, dtype: int64
PORT: 
Onvoldoende samples.
-------------------------
Tu

In [38]:
print(datasets_ddos)

['Friday-WorkingHours-Afternoon-DDoS', 'Friday-WorkingHours-Morning-Botnet', 'Wednesday-workingHours-DoS', 'cicdos2017', 'Botnet-Friday-02-03-2018_TrafficForML_CICFlowMeter', 'DoS2-Friday-16-02-2018_TrafficForML_CICFlowMeter', 'DoS1-Thursday-15-02-2018_TrafficForML_CICFlowMeter', 'DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter', 'DDoS2-Wednesday-21-02-2018_TrafficForML_CICFlowMeter', '01_12_DrDoS_DNS', '01_12_DrDoS_LDAP', '01_12_DrDoS_MSSQL', '01_12_DrDoS_NetBIOS', '01_12_DrDoS_NTP', '01_12_DrDoS_SNMP', '01_12_DrDoS_SSDP', '01_12_DrDoS_UDP', '01_12_Syn', '01_12_TFTP', '01_12_UDPLag', '03_11_Portmap', '03_11_Syn', '03_11_UDP', '03_11_UDPLag']


In [40]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import expand_dims
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras

cp_callbacks = [ tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', 
                    patience=20, 
                    min_delta=0.0001)
               ]

dataset = "DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter"
accuracies_dos = {}
results_dos = []
print('Start dataset:' + dataset)
for dataset_2 in datasets_ddos:
    if dataset == dataset_2  or dataset_2 == "DDoS2-Wednesday-21-02-2018_TrafficForML_CICFlowMeter":
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs_dos[dataset]
    for key in results_dos:
        dataframe.append(dfs_dos[key])
    dataframe.append(dfs_dos[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    loss, accuracy, f1_score, precision, recall = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
    if accuracy >= 0.92:
        results_dos.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(f1_score) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, balanced accuracy was ' + str(int(accuracy*10000)/100) + '%')
    accuracies_dos[dataset_2] = tuple([dataset_2, accuracy, loss])
dataframe = dfs_dos['DDoS2-Wednesday-21-02-2018_TrafficForML_CICFlowMeter']
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)

Start dataset:DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter
	proberen te combineren met Friday-WorkingHours-Afternoon-DDoS


ValueError: Found array with 0 sample(s) (shape=(0, 76)) while a minimum of 1 is required by MinMaxScaler.

## Accuracies

In [28]:
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([538666])
y_round = list(y_round)
print(len(y_round))

ValueError: cannot reshape array of size 291540 into shape (538666,)

In [29]:
from sklearn.metrics import accuracy_score
print("(Gebalanceerde) accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

(Gebalanceerde) accuracy van model met een ongekende dataset:


NameError: name 'y_true' is not defined

In [ ]:
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
accuracies_brute = {}
results_brute = []
print('Start dataset:' + dataset)
for dataset_2 in dfs_brute.keys():
    if dataset == dataset_2  or dataset_2 == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs_brute[dataset]
    for key in results_brute:
        dataframe.append(dfs_brute[key])
    dataframe.append(dfs_brute[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    loss, accuracy, f1_score, precision, recall = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
    if accuracy >= 0.92:
        results_brute.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(f1_score) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
    accuracies_cat[dataset_2] = tuple([dataset_2, accuracy, loss])
dataframe = dfs_brute['Wednesday-28-02-2018_TrafficForML_CICFlowMeter']
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([538666])
y_round = list(y_round)
print("(Gebalanceerde) accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

In [ ]:
dataset = "Wednesday-21-02-2018_TrafficForML_CICFlowMeter"
accuracies_port = {}
results_port = []
print('Start dataset:' + dataset)
for dataset_2 in dfs_port.keys():
    if dataset == dataset_2  or dataset_2 == "Wednesday-28-02-2018_TrafficForML_CICFlowMeter":
        continue
    print('\tproberen te combineren met ' + dataset_2)
    dataframe = dfs_port[dataset]
    for key in results_port:
        dataframe.append(dfs_port[key])
    dataframe.append(dfs_port[dataset_2])
    y = dataframe['label']
    scaler = MinMaxScaler()
    df = dataframe.drop(columns=['label'])
    X = scaler.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    loss, accuracy, f1_score, precision, recall = train_model(X_train, y_train, X_test, y_test, X_val, y_val)
    if accuracy >= 0.92:
        results_brute.append(dataset_2)
        print('\t\tgelukt! (nr.' + str(f1_score) + ') ' + str(int(accuracy*10000)/100) + '%')
    else:
        print('\t\t niet gelukt, accuracy was ' + str(int(accuracy*10000)/100) + '%')
    accuracies_port[dataset_2] = tuple([dataset_2, accuracy, loss])
dataframe = dfs_port['Wednesday-28-02-2018_TrafficForML_CICFlowMeter']
y_true = dataframe['label']
y_true = y_true.to_list()
scaler = MinMaxScaler()
df = dataframe.drop(columns=['label'])
X = scaler.fit_transform(df)
y = model.predict(X)
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([538666])
y_round = list(y_round)
print("(Gebalanceerde) accuracy van model met een ongekende dataset:")
print(str(int(accuracy_score(y_true, y_round)*10000)/100) + "%")

## Regularization
##### Activity Regularization: Penalize the model during training base on the magnitude of the activations.
##### Weight Constraint: Constrain the magnitude of weights to be within a range or below a limit.
##### Dropout: Probabilistically remove inputs during training.
##### Noise: Add statistical noise to inputs during training.
##### Early Stopping: Monitor model performance on a validation set and stop training when performance degrades.